In [0]:
import pandas as pd
import requests

headers ={
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept-Encoding': 'gzip, deflate,br',
    'Accept-Language': 'pt-BR, pt;q=0.9,en-US;q=0.8,en; q=0.7',
    'Connection': 'keep-alive',
}
url = "https://servicodados.ibge.gov.br/api/v3/agregados/4709/periodos/2022/variaveis/93|10605?localidades=N3[33,35]"

districts = requests.get(url).json()
###Conexão com API do IBGE, obtendo as informações de localidade do Rio de Janeiro e São Paulo

In [0]:
##trazendo os dados do JSON para vavariáveis do python
nome_municipio_rj= [municipios["resultados"][0]["series"][0]["localidade"]["nome"]  for municipios in districts_test]
print(nome_municipio_rj)
nome_municipio_sp= [municipios["resultados"][0]["series"][1]["localidade"]["nome"]  for municipios in districts_test]
print(nome_municipio_sp)
pop_municipio_rj= [municipios["resultados"][0]["series"][0]["serie"]["2022"]  for municipios in districts_test]
print(pop_municipio_rj)
pop_municipio_sp= [municipios["resultados"][0]["series"][1]["serie"]["2022"]  for municipios in districts_test]
print(pop_municipio_sp)
id_municipio_rj= [municipios["resultados"][0]["series"][0]["localidade"]["id"]  for municipios in districts_test]
print(id_municipio_rj)
id_municipio_sp= [municipios["resultados"][0]["series"][1]["localidade"]["id"]  for municipios in districts_test]
print(id_municipio_sp)


['Rio de Janeiro', 'Rio de Janeiro']
['São Paulo', 'São Paulo']
['16055174', '0.03']
['44411238', '0.61']
['33', '33']
['35', '35']


In [0]:
###transformando em pandas e formatando o dataframe com as informações do Rio de Janeiro
data_rj = pd.DataFrame.from_dict({'ID':id_municipio_rj, 'Nome':nome_municipio_rj, 'Populacao':pop_municipio_rj[0], 'TxCrescimento':pop_municipio_rj[1]})
data_rj

,ID,Nome,Populacao,TxCrescimento
0,33,Rio de Janeiro,16055174,0.03
1,33,Rio de Janeiro,16055174,0.03


In [0]:
###transformando em pandas e formatando o dataframe com as informações de São Paulo
data_sp = pd.DataFrame.from_dict({'ID': id_municipio_sp, 'Nome':nome_municipio_sp, 'Populacao':pop_municipio_sp[0], 'TxCrescimento':pop_municipio_sp[1]})
data_sp

,ID,Nome,Populacao,TxCrescimento
0,35,São Paulo,44411238,0.61
1,35,São Paulo,44411238,0.61


In [0]:
#unindo os dois dataframes e tirando as duplicidades.
union_df=pd.concat([data_sp,data_rj]).drop_duplicates()
union_df

,ID,Nome,Populacao,TxCrescimento
0,35,São Paulo,44411238,0.61
0,33,Rio de Janeiro,16055174,0.03


In [0]:
%sql
create database stage

/*CRIANDO UM database para separar o ambiente de stage e dw%/

In [0]:
%sql
create database dw

/*CRIANDO UM database para separar o ambiente de stage e dw%/

In [0]:
##criando um dataframe spark
df_spark=spark.createDataFrame(union_df)

df_spark.show()

+---+--------------+---------+-------------+
| ID|          Nome|Populacao|TxCrescimento|
+---+--------------+---------+-------------+
| 35|     São Paulo| 44411238|         0.61|
| 33|Rio de Janeiro| 16055174|         0.03|
+---+--------------+---------+-------------+



In [0]:
## criando uma view temporaria que será utilizada para gravação da stage.
df_spark.createOrReplaceTempView("df_ibge")

In [0]:
##carregando a stage a partir da view temporaria.
spark.sql(""" 
          create or replace table stage.df_ibge_stg
          using delta
          as select * from df_ibge
          """)

Out[24]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from stage.df_ibge_stg

ID,Nome,Populacao,TxCrescimento
35,São Paulo,44411238,0.61
33,Rio de Janeiro,16055174,0.03


In [0]:
##criação da dimensacao com as informações de codigo e descrição da localidade.
spark.sql(""" 
          create or replace table dw.dim_ibge
          using delta
          as select id, nome from df_ibge
          """)

Out[26]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from dw.dim_ibge


id,nome
35,São Paulo
33,Rio de Janeiro


In [0]:
spark.sql(""" 
          create or replace table dw.ft_ibge
          using delta
          as select id,  Populacao, TxCrescimento from df_ibge where 1=2
          """)

###criando a estrutura da tabela fato sem carga de dados.          

Out[47]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from dw.ft_ibge

id,Populacao,TxCrescimento


In [0]:
%sql

Select a.id, a.Populacao, a.TxCrescimento 
from stage.df_ibge_stg a inner join 
dw.dim_ibge b on a.id=b.id

/*testando os dados que serão carregados.*/

id,Populacao,TxCrescimento
35,44411238,0.61
33,16055174,0.03


In [0]:
data=("""insert into dw.ft_ibge Select a.id, a.Populacao, a.TxCrescimento 
          from stage.df_ibge_stg a inner join 
          dw.dim_ibge b on a.id=b.id""")
## variavel com o statement de insert          

In [0]:
spark.sql(data)
##inserindo os dados via sql spark.

Out[57]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from dw.ft_ibge

id,Populacao,TxCrescimento
35,44411238,0.61
33,16055174,0.03


In [0]:
%sql
select b.nome as Nome,sum(a.Populacao) as Populacao, sum(a.TxCrescimento) as Crescimento
          from stage.df_ibge_stg a inner join 
          dw.dim_ibge b on a.id=b.id
          group by b.nome

Nome,Populacao,Crescimento
São Paulo,4.4411238E7,0.61
Rio de Janeiro,1.6055174E7,0.03


Databricks visualization. Run in Databricks to view.